<a href="https://colab.research.google.com/github/Hidestament/AtCoder/blob/main/ABC/ABC194.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [AtCoder Beginner Contest 194](https://atcoder.jp/contests/abc194)

## [A - I Scream](https://atcoder.jp/contests/abc194/tasks/abc194_a)

In [ ]:
A, B = map(int, input().split())
if A + B >= 15 and B >= 8:
    print(1)
elif A + B >= 10 and B >= 3:
    print(2)
elif A + B >= 3:
    print(3)
else:
    print(4)

0 0
4


## [B - Job Assignment](https://atcoder.jp/contests/abc194/tasks/abc194_b)

### 全探索

In [ ]:
N = int(input())
jobs = [list(map(int, input().split())) for _ in range(N)]

3
8 5
4 4
7 9


In [ ]:
ans = 10**10
for i in range(N):
    for j in range(N):
        if i == j:
            temp = jobs[i][0] + jobs[j][1]
        else:
            temp = max(jobs[i][0], jobs[j][1])

        ans = min(ans, temp)
print(ans)

5


### O(N)解法

In [ ]:
N = int(input())
A = []
B = []
for _ in range(N):
    a, b = map(int, input().split())
    A.append(a)
    B.append(b)

3
8 5
4 4
7 9


In [ ]:
min_a_ind = A.index(min(A))
min_b_ind = B.index(min(B))

In [ ]:
if min_a_ind == min_b_ind:
    ans = 10**10
    for i in range(N):
        if i != min_a_ind:
            ans = min(ans, max(A[min_a_ind], B[i]), max(A[i], B[min_b_ind]))
    ans = min(ans, min(A) + min(B))
else:
    ans = max(min(A), min(B))

5
7


In [ ]:
print(ans)

## [C - Squared Error](https://atcoder.jp/contests/abc194/tasks/abc194_c)

### 式変形によるO(N)解法

In [ ]:
from itertools import accumulate
N = int(input())
A = list(map(int, input().split()))

3
2 8 4


In [ ]:
ans = 0
for i in range(N):
    ans += (A[i] ** 2)
ans *= N

In [ ]:
s = sum(A)
for i in range(N):
    ans -= A[i] * s

In [ ]:
print(ans)

56

簡潔にかくと

In [ ]:
from itertools import accumulate
N = int(input())
A = list(map(int, input().split()))

In [ ]:
ans = N * sum(A[i] ** 2 for i in range(N))
s = sum(A)
ans = ans - sum(A[i] * s for i in range(N))
print(ans)

### |A| <= 200を利用する解法

In [ ]:
from collections import Counter
N = int(input())
A = list(map(int, input().split()))
cnt = Counter(A)

In [ ]:
ans = 0
for key1, value1 in cnt.items():
    for key2, value2 in cnt.items():
        ans += (key1 - key2) ** 2 * value1 * value2
print(ans // 2)

## [D - Journey](https://atcoder.jp/contests/abc194/tasks/abc194_d)

In [ ]:
N = int(input())

2


In [ ]:
ans = N * sum(1/i for i in range(1, N))
print(ans)

2.0


## [E - Mex Min](https://atcoder.jp/contests/abc194/tasks/abc194_e)

### 解法1: heapqで頑張る（自力）

In [ ]:
from collections import deque
from heapq import heapify, heappop, heappush
N, M = map(int, input().split())
A = list(map(int, input().split()))

7 3
0 0 1 2 0 1 0


In [ ]:
hq = [i for i in range(N+10)]
heapify(hq)

In [ ]:
from collections import deque
used = [0] * (N+1)
exist = set([i for i in range(N+10)])
dq = deque()
ans = 10**10
for l in range(N):
    a = A[l]
    used[a] += 1
    dq.append(a)

    if len(dq) > M:
        res = dq.popleft()
        used[res] -= 1
        if used[res] == 0 and res not in exist:
            heappush(hq, res)

    if len(dq) == M:
        while True:
            minimum = heappop(hq)
            if used[minimum] == 0:
                break
            else:
                exist.discard(minimum)

        ans = min(ans, minimum)

In [ ]:
print(ans)

2

### 解法2

In [ ]:
from collections import deque
N, M = map(int, input().split())
A = list(map(int, input().split()))
dq = deque()
used = [0] * (N+10)

7 3
0 0 1 2 0 1 0


In [ ]:
# 最初の 0 ~ M の区間に対するmexを求める
ans = 10**10
dq = deque()
for i in range(M):
    a = A[i]
    dq.append(a)
    used[a] += 1

for i in range(N+10):
    if used[i] == 0:
        ans = min(ans, i)

In [ ]:
# その後の区間の変化に対するmexを求める
for i in range(M, N):
    a = A[i]
    dq.append(a)
    res = dq.popleft()
    used[a] += 1
    used[res] -= 1
    if used[res] == 0:
        ans = min(ans, res)

In [ ]:
print(ans)

2


### 解答3: Seg木で頑張る

In [62]:
class SegmentTree:
    """ 1点更新・区間集約 Segment Tree. 非再起・1-index・非可換

    Attributes:
        tree (List[Union[int, float]]): セグメントツリーを表す配列
        segfunc (Callable): 集約を行う関数 (モノイド上の演算)
        ide_ele (float): 集約の関数における単位元　（モノイド上の単位元）
        update (int, float): 元の配列の更新 & セグメントツリーの更新
        query (int, int): 区間集約値の取得
    """
    def __init__(self, n: int, ide_ele: float, seg_func: callable):
        """
        Args:
            n (int): 元の配列のサイズ
            ide_ele (float): モノイドにおける単位元
            segfunc (callable): 集約を行う関数 （モノイド上の演算）
        """
        self.segfunc = seg_func
        self.ide_ele = ide_ele
        self.num = 1 << (n - 1).bit_length()
        self.tree = [self.ide_ele] * (2 * self.num)
    
    def update(self, i: int, x: float) -> None:
        """元の配列のi番目を x に変更（上書き）する. それに伴ってセグメントツリーも更新する. +=のようなupdateではない.

        Args:
            i (int): 元の配列のi番目の要素. 元の配列の0-indexの
            x (float): 変更後の値        
        """
        i += self.num
        self.tree[i] = x
        while i > 1:
            i >>= 1
            self.tree[i] = self.segfunc(self.tree[i << 1], self.tree[(i << 1) + 1])
        
    def query(self, l: int, r: int):
        """元の配列の閉区間[l, r]のsegfuncにおける集約値を取得する

        Args:
            l (int): 元の配列の集約を行う区間の左端のindex. 0-index
            r (int): 元の配列の集約を行う区間の右端のindex. 0-index
        """
        # [l, r) として葉から計算していく
        l += self.num
        r += self.num + 1

        l_res = self.ide_ele
        r_res = self.ide_ele
        
        while l < r:
            if l & 1:  # lが奇数だったらそのノードを集約する
                l_res = self.segfunc(l_res, self.tree[l])
                l += 1  # 右に進む
            if r & 1:  # rが奇数だったら, rは開区間なので その1つ左ノードを集約する
                r_res = self.segfunc(self.tree[r - 1], r_res)

            l >>= 1  # 上に進む（２で割って切り捨て）
            r >>= 1
        
        return self.segfunc(l_res, r_res)

In [67]:
from collections import deque
N, M = map(int, input().split())
A = list(map(int, input().split()))
cnt = [0] * (N + 10)

3 2
0 0 1


In [68]:
seg = SegmentTree(N+10, 10**15, min)
for i in range(N+10):
    seg.update(i, i)

In [69]:
INF = 10**15
ans = INF
for i in range(N):
    cnt[A[i]] += 1
    seg.update(A[i], INF)
    if i >= M:
        a = A[i - M]
        cnt[a] -= 1
        if cnt[a] == 0:
            seg.update(a, a)

    if i >= M - 1:
        ans = min(ans, seg.query(0, N + 10))
print(ans)

1
